<a href="https://colab.research.google.com/github/vektor8891/llm/blob/main/projects/11_bert_pretraining/11_bert_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pretraining a BERT model

## Loading data

In [1]:
!wget -O BERT_dataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/bZaoQD52DcMpE7-kxwAG8A.zip
!unzip BERT_dataset.zip

--2025-04-24 15:13:19--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/bZaoQD52DcMpE7-kxwAG8A.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88958506 (85M) [application/zip]
Saving to: ‘BERT_dataset.zip’

BERT_dataset.zip    100%[===================>]  84.84M  23.7MB/s    in 3.9s    

2025-04-24 15:13:23 (22.0 MB/s) - ‘BERT_dataset.zip’ saved [88958506/88958506]

Archive:  BERT_dataset.zip
   creating: bert_dataset/
  inflating: bert_dataset/.DS_Store  
  inflating: bert_dataset/bert_train_data.csv  
  inflating: bert_dataset/bert_test_data_sampled.csv  
  inflating: bert_dataset/bert_test_data.csv  
  inflating: bert_dataset/bert_train_data_s

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import pandas as pd
import json


class BERTCSVDataset(Dataset):
    def __init__(self, filename):
        self.data = pd.read_csv(filename)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        try:

            bert_input = torch.tensor(json.loads(row['BERT Input']), dtype=torch.long)
            bert_label = torch.tensor(json.loads(row['BERT Label']), dtype=torch.long)
            segment_label = torch.tensor([int(x) for x in row['Segment Label'].split(',')], dtype=torch.long)
            is_next = torch.tensor(row['Is Next'], dtype=torch.long)
            original_text = row['Original Text']  # If you want to use it
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for row {idx}: {e}")
            print("BERT Input:", row['BERT Input'])
            print("BERT Label:", row['BERT Label'])
            # Handle the error, e.g., by skipping this row or using default values
            return None  # or some default values

            # Tokenizing the original text with BERT
        encoded_input = self.tokenizer.encode_plus(
            original_text,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encoded_input['input_ids'].squeeze()
        attention_mask = encoded_input['attention_mask'].squeeze()

        return(bert_input, bert_label, segment_label, is_next, input_ids, attention_mask, original_text)

In [10]:
from torch.nn.utils.rnn import pad_sequence

# create a collate function that applies transformations on batches of data iterator
PAD_IDX = 0
def collate_batch(batch):


    bert_inputs_batch, bert_labels_batch, segment_labels_batch, is_nexts_batch,input_ids_batch,attention_mask_batch,original_text_battch = [], [], [], [],[],[],[]

    for bert_input, bert_label, segment_label, is_next,input_ids,attention_mask,original_text in batch:
        # Convert each sequence to a tensor and append to the respective list
        bert_inputs_batch.append(bert_input.clone().detach())
        bert_labels_batch.append(bert_label.clone().detach())
        segment_labels_batch.append(segment_label.clone().detach())
        is_nexts_batch.append(is_next)
        input_ids_batch.append(input_ids)
        attention_mask_batch.append(attention_mask)
        original_text_battch.append(original_text)

    # Pad the sequences in the batch
    bert_inputs_final = pad_sequence(bert_inputs_batch, padding_value=PAD_IDX, batch_first=False)
    bert_labels_final = pad_sequence(bert_labels_batch, padding_value=PAD_IDX, batch_first=False)
    segment_labels_final = pad_sequence(segment_labels_batch, padding_value=PAD_IDX, batch_first=False)
    is_nexts_batch = torch.tensor(is_nexts_batch, dtype=torch.long)

    return bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_batch

In [11]:
# create train and test dataloaders
BATCH_SIZE = 2

train_dataset_path = './bert_dataset/bert_train_data.csv'
test_dataset_path = './bert_dataset/bert_test_data.csv'

train_dataset = BERTCSVDataset(train_dataset_path)
test_dataset = BERTCSVDataset(test_dataset_path)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

## Model creation

3 types of embeddings used in BERT to represent input tokens:

1. Token Embedding: initial representation of each token
2. Positional Embedding: captures the order of tokens
3. Segment Embedding: differentiates between different segments (e.g. sentences)

Model components:

1. Initialization: subclass of `torch.nn.Module`
2. Embedding Layer: combines token embeddings and segment embeddings
3. Transformer Encoder: encodes the input embeddings
4. Next Sentence Prediction: predicts the relationship between two consecutive sentences using the output of Transformer encoder
5. Masked Language Modeling: predicts the masked tokens in the input sequence
6. Forward Pass: defines the forward pass. Returns predictions for Next Sentence Prediction and Masked Language Modeling using input tokens and segment labels


In [24]:
import torch.nn as nn
from torch import Tensor
import math

EMBEDDING_DIM = 10

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Define the PositionalEncoding class as a PyTorch module for adding positional information to token embeddings
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        # Create a positional encoding matrix as per the Transformer paper's formula
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        # Apply the positional encodings to the input token embeddings

        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class BERTEmbedding (nn.Module):

    def __init__(self, vocab_size, emb_size ,dropout=0.1,train=True):

        super().__init__()

        self.token_embedding = TokenEmbedding( vocab_size,emb_size )
        self.positional_encoding = PositionalEncoding(emb_size,dropout)
        self.segment_embedding = nn.Embedding(3, emb_size)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, bert_inputs, segment_labels=False):
        my_embeddings=self.token_embedding(bert_inputs)
        if self.train:
          x = self.dropout(my_embeddings + self.positional_encoding(my_embeddings) + self.segment_embedding(segment_labels))
        else:
          x = my_embeddings + self.positional_encoding(my_embeddings)

        return x

In [14]:
# example
VOCAB_SIZE=147161
batch = 2
count = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# load sample batches from dataloader
for batch in train_dataloader:
    bert_inputs, bert_labels, segment_labels, is_nexts = [b.to(device) for b in batch]
    count += 1
    if count == 5:
        break

print(bert_inputs.shape)
print(bert_inputs[:,0])
print(segment_labels.shape)
print(segment_labels[:,0])

torch.Size([102, 2])
tensor([    1,     3,  2929,    13,   103,    11,  8245, 44449,    51,     5,
         3455,  9236,    12,    19,  3545,     3, 15441,    31,     3,  2665,
            7,   599,    13,  1638,     6,     2,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,    47,     7,     5,  2428,    15,     5,  2541,   213,     7,
           15,     3,   946,    10,     3, 13541,    28,    10,     3,  3769,
         2541,     3,     8,   240,   335,    12,    19,     3, 14294,    27,
            7,    13,     9,  2644,  3555,     7,   383,  1573,   156,    25,
           32,   171, 15549,    43,     5, 18301, 10077,     3,     5,   527,
            6,     2])
torch.Size([102, 2])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [15]:
# Instantiate the TokenEmbedding
token_embedding = TokenEmbedding(VOCAB_SIZE, emb_size=EMBEDDING_DIM )

# Get the token embeddings for a sample input
t_embeddings = token_embedding(bert_inputs)
# Each token is transformed into a tensor of size emb_size
print(f"Dimensions of token embeddings: {t_embeddings.size()}") # Expected: (sequence_length, batch_size, EMBEDDING_DIM)
# Check the embedded vectors for first 3 tokens of the first sample in the batch
# you get embeddings[i,0,:] where i refers to the i'th token of the first sample in the batch (b=0)
for i in range(3):
    print(f"Token Embeddings for the {i}th token of the first sample: {t_embeddings[i,0,:]}")

Dimensions of token embeddings: torch.Size([102, 2, 10])
Token Embeddings for the 0th token of the first sample: tensor([-0.1569, -0.0639, -1.7253,  1.4022,  4.4875, -1.4307, -2.1106,  2.2914,
         0.6288,  0.9770], grad_fn=<SliceBackward0>)
Token Embeddings for the 1th token of the first sample: tensor([ 0.0359,  2.3731,  4.1027, -3.1912, -4.3995,  1.3404, -5.1265, -3.3133,
        -2.1321, -6.6937], grad_fn=<SliceBackward0>)
Token Embeddings for the 2th token of the first sample: tensor([-1.5487, -5.3569,  0.9062,  2.2734,  0.6782, -3.8600, -0.8670,  1.4534,
        -2.2636, -4.3005], grad_fn=<SliceBackward0>)


In [18]:
positional_encoding = PositionalEncoding(emb_size=EMBEDDING_DIM,dropout=0)

# Apply positional encoding to token embeddings
p_embedding = positional_encoding(t_embeddings)

print(f"Dimensions of positionally encoded tokens: {p_embedding.size()}") # Expected: (sequence_length, batch_size, EMBEDDING_DIM)
# Check the positional encoded vectors for first 3 tokens of the first sample in the batch
# you get encoded_tokens[i,0,:] where i refers to the i'th token of the first sample(b=0) in the batch
for i in range(3):
    print(f"Positional Embeddings for the {i}th token of the first sample: {p_embedding[i,0,:]}")

Dimensions of positionally encoded tokens: torch.Size([102, 2, 10])
Positional Embeddings for the 0th token of the first sample: tensor([-0.1569,  0.9361, -1.7253,  2.4022,  4.4875, -0.4307, -2.1106,  3.2914,
         0.6288,  1.9770], grad_fn=<SliceBackward0>)
Positional Embeddings for the 1th token of the first sample: tensor([ 0.8774,  2.9134,  4.2605, -2.2037, -4.3743,  2.3401, -5.1225, -2.3134,
        -2.1315, -5.6937], grad_fn=<SliceBackward0>)
Positional Embeddings for the 2th token of the first sample: tensor([-0.6394, -5.7731,  1.2179,  3.2236,  0.7285, -2.8613, -0.8590,  2.4534,
        -2.2623, -3.3005], grad_fn=<SliceBackward0>)


In [16]:
segment_embedding = nn.Embedding(3, EMBEDDING_DIM)
s_embedding = segment_embedding(segment_labels)
print(f"Dimensions of segment embedding: {s_embedding.size()}") # Expected: (sequence_length, batch_size, EMBEDDING_DIM)
# Check the Segment Embedding vectors for first 3 tokens of the first sample in the batch
# you get segment_embedded[i,0,:] where i refers to the i'th token of the first sample(b=0) in the batch
for i in range(3):
    print(f"Segment Embeddings for the {i}th token of the first sample: {s_embedding[i,0,:]}")

Dimensions of segment embedding: torch.Size([102, 2, 10])
Segment Embeddings for the 0th token of the first sample: tensor([ 0.3030, -0.3011,  0.5643,  0.4439, -0.8016, -0.4365, -0.0510, -1.6664,
         0.9397,  2.7754], grad_fn=<SliceBackward0>)
Segment Embeddings for the 1th token of the first sample: tensor([ 0.3030, -0.3011,  0.5643,  0.4439, -0.8016, -0.4365, -0.0510, -1.6664,
         0.9397,  2.7754], grad_fn=<SliceBackward0>)
Segment Embeddings for the 2th token of the first sample: tensor([ 0.3030, -0.3011,  0.5643,  0.4439, -0.8016, -0.4365, -0.0510, -1.6664,
         0.9397,  2.7754], grad_fn=<SliceBackward0>)


In [19]:
# Create the combined embedding vectors
bert_embeddings = t_embeddings + p_embedding + s_embedding
print(f"Dimensions of token + position + segment encoded tokens: {bert_embeddings.size()}")
#Check the BERT Embedding vectors for first 3 tokens of the first sample in the batch
# you get bert_embeddings[i,0,:] where i refers to the i'th token of the first sample(b=0) in the batch
for i in range(3):
    print(f"BERT_Embedding for {i}th token: {bert_embeddings[i,0,:]}")

Dimensions of token + position + segment encoded tokens: torch.Size([102, 2, 10])
BERT_Embedding for 0th token: tensor([-0.0108,  0.5711, -2.8862,  4.2483,  8.1734, -2.2979, -4.2722,  3.9164,
         2.1972,  5.7294], grad_fn=<SliceBackward0>)
BERT_Embedding for 1th token: tensor([  1.2163,   4.9854,   8.9276,  -4.9510,  -9.5754,   3.2440, -10.3000,
         -7.2931,  -3.3239,  -9.6120], grad_fn=<SliceBackward0>)
BERT_Embedding for 2th token: tensor([ -1.8851, -11.4311,   2.6883,   5.9409,   0.6051,  -7.1578,  -1.7770,
          2.2404,  -3.5863,  -4.8256], grad_fn=<SliceBackward0>)


In [22]:
class BERT(torch.nn.Module):

    def __init__(self, vocab_size, d_model=768, n_layers=12, heads=12, dropout=0.1):
        """
        vocab_size: The size of the vocabulary.
        d_model: The size of the embeddings (hidden size).
        n_layers: The number of Transformer layers.
        heads: The number of attention heads in each Transformer layer.
        dropout: The dropout rate applied to embeddings and Transformer layers.
        """
        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads

        # Embedding layer that combines token embeddings and segment embeddings
        self.bert_embedding = BERTEmbedding(vocab_size, d_model, dropout)

        # Transformer Encoder layers
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=heads, dropout=dropout,batch_first=False)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)

        # Linear layer for Next Sentence Prediction
        self.nextsentenceprediction = nn.Linear(d_model, 2)

        # Linear layer for Masked Language Modeling
        self.masked_language = nn.Linear(d_model, vocab_size)

    def forward(self, bert_inputs, segment_labels):
        """
        bert_inputs: Input tokens.
        segment_labels: Segment IDs for distinguishing different segments in the input.
        mask: Attention mask to prevent attention to padding tokens.

        return: Predictions for next sentence task and masked language modeling task.
        """

        padding_mask = (bert_inputs == PAD_IDX).transpose(0, 1)
        # Generate embeddings from input tokens and segment labels
        my_bert_embedding = self.bert_embedding(bert_inputs, segment_labels)

        # Pass embeddings through the Transformer encoder
        transformer_encoder_output = self.transformer_encoder(my_bert_embedding,src_key_padding_mask=padding_mask)


        next_sentence_prediction = self.nextsentenceprediction(transformer_encoder_output[ 0,:])


        # Masked Language Modeling: Predict all tokens in the sequence
        masked_language = self.masked_language(transformer_encoder_output)

        return  next_sentence_prediction, masked_language

In [25]:
# create an instance of the model
EMBEDDING_DIM = 10

# Define parameters
vocab_size = 147161  # Replace VOCAB_SIZE with your vocabulary size
d_model = EMBEDDING_DIM  # Replace EMBEDDING_DIM with your embedding dimension
n_layers = 2  # Number of Transformer layers
initial_heads = 12 # Initial number of attention heads
initial_heads = 2
# Ensure the number of heads is a factor of the embedding dimension
heads = initial_heads - d_model % initial_heads

dropout = 0.1  # Dropout rate

# Create an instance of the BERT model
model = BERT(vocab_size, d_model, n_layers, heads, dropout)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [26]:
padding_mask = (bert_inputs == PAD_IDX).transpose(0, 1)
padding_mask.shape

torch.Size([2, 102])

In [27]:
encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=heads, dropout=dropout,batch_first=False)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
# Pass embeddings through the Transformer encoder
transformer_encoder_output = transformer_encoder(bert_embeddings,src_key_padding_mask=padding_mask)
transformer_encoder_output.shape

torch.Size([102, 2, 10])

In [28]:
nextsentenceprediction = nn.Linear(d_model, 2)
nsp = nextsentenceprediction(transformer_encoder_output[ 0,:])
#logits for NSP task
print(f"NSP Output Shape: {nsp.shape}")  # Expected shape: (batch_size, 2)

NSP Output Shape: torch.Size([2, 2])


In [29]:
masked_language = nn.Linear(d_model, vocab_size)
# Masked Language Modeling: Predict all tokens in the sequence
mlm = masked_language(transformer_encoder_output)
#logits for MLM task
print(f"MLM Output Shape: {mlm.shape}")  # Expected shape: (seq_length, batch_size, vocab_size)

MLM Output Shape: torch.Size([102, 2, 147161])
